In [1]:
import pandas as pd
import json, requests
import math
import numpy as np
from pathlib import Path
from collections import defaultdict

## Setup

In [2]:
data_folder = Path('../../data/county')

In [3]:
code_to_state = {'01': 'Alabama',
 '02': 'Alaska',
 '04': 'Arizona',
 '05': 'Arkansas',
 '06': 'California',
 '08': 'Colorado',
 '09': 'Connecticut',
 '10': 'Delaware',
 '11': 'District of Columbia',
 '12': 'Florida',
 '13': 'Georgia',
 '15': 'Hawaii',
 '16': 'Idaho',
 '17': 'Illinois',
 '18': 'Indiana',
 '19': 'Iowa',
 '20': 'Kansas',
 '21': 'Kentucky',
 '22': 'Louisiana',
 '23': 'Maine',
 '24': 'Maryland',
 '25': 'Massachusetts',
 '26': 'Michigan',
 '27': 'Minnesota',
 '28': 'Mississippi',
 '29': 'Missouri',
 '30': 'Montana',
 '31': 'Nebraska',
 '32': 'Nevada',
 '33': 'New Hampshire',
 '34': 'New Jersey',
 '35': 'New Mexico',
 '36': 'New York',
 '37': 'North Carolina',
 '38': 'North Dakota',
 '39': 'Ohio',
 '40': 'Oklahoma',
 '41': 'Oregon',
 '42': 'Pennsylvania',
 '44': 'Rhode Island',
 '45': 'South Carolina',
 '46': 'South Dakota',
 '47': 'Tennessee',
 '48': 'Texas',
 '49': 'Utah',
 '50': 'Vermont',
 '51': 'Virginia',
 '53': 'Washington',
 '54': 'West Virginia',
 '55': 'Wisconsin',
 '56': 'Wyoming',
 '72': 'Puerto Rico'}

In [4]:
state_to_abbr = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [5]:
with open(data_folder/"counties20m.json", 'r') as f:
    county_json = json.load(f)

## Alter Key Names 

This section is to make key names lowercases and consistent

In [6]:
keys = ['GEO_ID', 'COUNTY', 'NAME', 'LSAD', 'CENSUSAREA', 'STATE']

In [7]:
for county in county_json["features"]:
    for key in keys:
        county["properties"][key.lower()] = county["properties"][key]
        del county["properties"][key]

In [8]:
county_json["features"][0]

{'type': 'Feature',
 'properties': {'geo_id': '0500000US01001',
  'county': '001',
  'name': 'Autauga',
  'lsad': 'County',
  'censusarea': 594.436,
  'state': '01'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]}}

## Add State Name

In [9]:
for cur_county in county_json["features"]:
    cur_county["properties"]["statename"] = code_to_state[cur_county["properties"]["state"]]
    cur_county["properties"]["stateabbr"] = state_to_abbr[cur_county["properties"]["statename"]]

## Add population data

In [10]:
alt_names = {"Wade Hampton":"Kusilvak Census Area"}

In [11]:
df_pop = pd.read_csv(data_folder/"population.csv")

In [12]:
d = {}
for row in df_pop.iterrows():
    d[row[1].STATE] = row[1].STNAME

In [13]:
df_pop = df_pop[["STNAME", "CTYNAME", "POPESTIMATE2018", "STATE", "COUNTY"]]

In [14]:
for cur_county in county_json["features"]:
    props = cur_county["properties"]
    state = props["state"]
    county = props["county"]
    pop = df_pop[(df_pop["STATE"] == int(state))&(df_pop["COUNTY"] == int(county))]["POPESTIMATE2018"]
    if(len(pop.values) > 0):
        props["population"] = pop.values[0]
    else:
        if(props["statename"] == "Virginia" and props["name"] == "Bedford"): props["population"] = 78742
        elif(props["statename"] == "South Dakota" and props["name"] == "Shannon"): props["population"] = 14309
        else: print(props["statename"], props["name"], "not found")

Alaska Wade Hampton not found
Puerto Rico Arecibo not found
Puerto Rico Barranquitas not found
Puerto Rico Caguas not found
Puerto Rico Carolina not found
Puerto Rico Comerío not found
Puerto Rico Florida not found
Puerto Rico Guaynabo not found
Puerto Rico Humacao not found
Puerto Rico Las Marías not found
Puerto Rico Maricao not found
Puerto Rico Naranjito not found
Puerto Rico Quebradillas not found
Puerto Rico San Lorenzo not found
Puerto Rico Toa Alta not found
Puerto Rico Vega Baja not found
Puerto Rico Camuy not found
Puerto Rico Cayey not found
Puerto Rico Cidra not found
Puerto Rico Coamo not found
Puerto Rico Fajardo not found
Puerto Rico Guánica not found
Puerto Rico Guayanilla not found
Puerto Rico Hormigueros not found
Puerto Rico Jayuya not found
Puerto Rico Aguadilla not found
Puerto Rico Lajas not found
Puerto Rico Lares not found
Puerto Rico Luquillo not found
Puerto Rico Maunabo not found
Puerto Rico Morovis not found
Puerto Rico Naguabo not found
Puerto Rico Patillas

### Add Missing Alaska County and Puerto Rico Data

In [15]:
df_pop_add = pd.read_csv(data_folder/"population2.csv")

In [16]:
pop_dict = {row[1]: row[2] for row in df_pop_add.values}

In [17]:
for cur_county in county_json["features"]:
    props = cur_county["properties"]
    geo_id = props["geo_id"]
    if(geo_id in pop_dict):
        props["population"] = pop_dict[geo_id]

### Alert to any missing populations

In [18]:
for cur_county in county_json["features"]:
    try:
        pop = int(cur_county["properties"]["population"])
    except KeyError:
        print("No population found for ", cur_county["properties"])
        raise KeyError
    except ValueError:
        print(cur_county["properties"]["population"], "is not a valid population")
        raise ValueError

## Add Land Area in Square Miles

In [19]:
df_land_area = pd.read_csv(data_folder/"countyArea.csv", encoding = "ISO-8859-1")

In [20]:
df_land_area = df_land_area[["GCT_STUB.target-geo-id", "SUBHD0303", "HD02"]]

In [21]:
df_land_area = df_land_area.rename(columns={"GCT_STUB.target-geo-id":"geo_id", "SUBHD0303":"land_area", "HD02":"housing_units"})

In [22]:
len("0500000US01001")

14

In [23]:
df_land_area.head()

,geo_id,land_area,housing_units
0,Target Geo Id,Area in square miles - Land area,Housing units
1,0100000US,3531905.43,131704730
2,0400000US01,50645.33,2171853
3,0500000US01001,594.44,22135
4,0500000US01003,1589.78,104061


In [24]:
# get only the rows that are counties (some are states and 1 is US overall)
df_land_area = df_land_area[df_land_area["geo_id"].map(len) == 14]

In [25]:
for cur_county in county_json["features"]:
    props = cur_county["properties"]
    county_id = props["geo_id"]
    _, land_area, housing_units = df_land_area[df_land_area["geo_id"]==county_id].values[0]
    props["land_area"] = land_area
    props["housing_units"] = housing_units
    props["population_density"] = int(props["population"])/float(land_area)

In [26]:
county_json["features"][0]["properties"]

{'geo_id': '0500000US01001',
 'county': '001',
 'name': 'Autauga',
 'lsad': 'County',
 'censusarea': 594.436,
 'state': '01',
 'statename': 'Alabama',
 'stateabbr': 'AL',
 'population': 55601,
 'land_area': '594.44',
 'housing_units': '22135',
 'population_density': 93.53509185115402}

## Add Covid Data

In [27]:
api_url = "https://covidcompare.com/data.json"
county_covid = requests.get(api_url).json()

In [28]:
# #these are the things that appear in 'county' for which we should delete the entire record
deletions = ['Unassigned', 'Unknown','Waiting on information','Ormsby','Southwest Utah','Non-Utah resident','Non-Utah Resident',
             'Non-Florida-Resident','Outside-Florida','Sheridan County','Oklahoma Out-Of-State']

subs = {
    'Weber-Morgan':'Weber',
    'Virginia Beach City':'Virginia Beach',
    'Alexandria City (VA)':'Alexandria',
    'Alexandria City':'Alexandria',
    'Charlottesville City':'Charlottesville',
    'Laporte':'LaPorte',
    'Dupage':'DuPage',
    'Mccook':'McCook',
    'Mchenry':'McHenry',
    'Fond Du Lac':'Fond du Lac',
    '?Benton':'Benton',
    'Desoto':'DeSoto',
    'Bear River':'Box Elder',
    'St. John The Baptist':'St. John the Baptist',
    'Prince George\'S':'Prince George\'s',
    'Hawaii Island':'Hawaii',
    'Ketchikan Gateway Borough':'Ketchikan Gateway',
    'Mclean':'McLean',
    'Lasalle':'LaSalle',
    'Dade':'Miami-Dade',
    'Richmond City':'Richmond',
    'Isle Of Wight':'Isle of Wight',
    'Portsmouth City':'Portsmouth',
    'Detroit City':'Wayne',
    'Hay':'Hays',
    'Mclennan':'McLennan',
    'Dewitt':'DeWitt',
    'Elko County':'Elko',
    'Filmore':'Fillmore',
    'Stanley':'Stanly',
    'Lewis And Clark':'Lewis and Clark',
    'Berknap':'Belknap',
    'Kansas City':'Jackson',
    'Delaware County':'Delaware',
    'Navajo Nation':'Navajo',
    'Mckinley':'McKinley',
    'Seward':'Kenai Peninsula',
    'Mcclain':'McClain',
    '​Benton':'Benton',
    'Blue earth':'Blue Earth',
    'Washington, D.C.':'District of Columbia',
    
}

doubles = {
    ('Maryland', 'Baltimore'):['Baltimore', 'Baltimore City'],
    
    ('Missouri', 'Jackson'):['Kansas City', 'Jackson'],
    ('Missouri', 'St. Louis'):['St. Louis', 'St. Louis City'],
    #('Texas','Harris'):['Harris-Non Houston', 'Harris-Houston'], 
}

In [29]:
for (state,county), matches in doubles.items():
    total_cases, total_deaths = (0, 0)
    for match in matches:
        case_dict = county_covid[state].pop(match)
        total_cases += int(case_dict["confirmed"])
        total_deaths += int(case_dict["deaths"])
    county_covid[state][county] = {"confirmed":str(total_cases), "deaths":str(total_deaths)}

clean_dict = {}

In [30]:
for state, county_dict in county_covid.items():
    if state == 'Washington, D.C.': state = "District of Columbia" 
    clean_dict[state] = {}
    for county, data in county_dict.items():
#         if(county in deletions): continue
        if(county in subs): 
            new_name = subs[county]
            clean_dict[state][new_name] = county_dict[county]
        else:
            clean_dict[state][county] = county_dict[county]

In [31]:
def flatten_json_to_records(json_dict):
    records = []
    for state, county_dict in clean_dict.items():
        for county, data_dict in county_dict.items():
            deaths = data_dict["deaths"]
            cases = data_dict["confirmed"]
            records.append((state.strip(), county.strip(), int(cases.strip()), int(deaths.strip())))
    return records

In [32]:
case_records = flatten_json_to_records(clean_dict)

In [33]:
codes = defaultdict(dict)
for cur_county in county_json["features"]:
    countyID = cur_county["properties"]["geo_id"]
    code = cur_county["properties"]["state"]
    countyName = cur_county["properties"]["name"]
    state = code_to_state[code]
    codes[state][countyName] = countyID

In [34]:
def get_county_cases():
    dict_cases = {}
    used_states = []
    for (state, county, cases, deaths) in case_records:
        if not state in used_states:
            for county_temp, geo in codes[state].items():
                dict_cases[geo] = (state, county_temp, 0, 0)
            used_states.append(state)
        try:
            geo_id = codes[state][county]
            dict_cases[geo_id] = (state, county, cases, deaths)
        except KeyError:
            print(f"County not found:'{state}', '{county}', '{cases}'")
    return dict_cases

In [35]:
codes["New York"]["Richmond"]

'0500000US36085'

In [36]:
dict_cases = get_county_cases()

County not found:'New York', 'Unassigned', '23'
County not found:'Washington', 'Unassigned', '159'
County not found:'Washington', 'Walla Walla County', '1'
County not found:'New Jersey', 'Unassigned', '221'
County not found:'Illinois', 'Unassigned', '2'
County not found:'Michigan', 'Out of State', '2'
County not found:'Florida', 'Unassigned', '1'
County not found:'Louisiana', 'Unassigned', '50'
County not found:'Massachusetts', 'Unassigned', '30'
County not found:'Massachusetts', 'Dukes and Nantucket', '1'
County not found:'Georgia', 'Unassigned', '27'
County not found:'Colorado', 'Unassigned', '1'
County not found:'Tennessee', 'Out-of-state', '82'
County not found:'Tennessee', 'Unknown', '30'
County not found:'Nevada', 'Unassigned', '36'
County not found:'Utah', 'Non-Utah resident', '12'
County not found:'Utah', 'Southwest Utah', '1'
County not found:'Minnesota', 'LeSeur', '1'
County not found:'Arkansas', 'Unassigned', '121'
County not found:'Missouri', 'Unassigned', '2'
County not fo

In [37]:
subs = {
    '0500000US51760':'0500000US51159', 
    '0500000US51600':'0500000US51059',
    '0500000US51019':'0500000US51515',
    '0500000US51067':'0500000US51620',
    '0500000US51161':'0500000US51770',
    '0500000US2a4005':'0500000US24510',
    '0500000US29510':'0500000US29189',
}
for cur_county in county_json["features"]:
    county_id = cur_county["properties"]["geo_id"]
    if county_id in dict_cases:
        cur_county["properties"]["cases"] = dict_cases[county_id][2]
        cur_county["properties"]["deaths"] = dict_cases[county_id][3]
    else:
        if (county_id in subs):
            correct_id = subs[county_id]
            print(f"Subbing in {correct_id} for {county_id}")
            cur_county["properties"]["cases"] = dict_cases[correct_id][2]
            cur_county["properties"]["deaths"] = dict_cases[correct_id][3]
        else:
            cur_county["properties"]["cases"] = 0
            cur_county["properties"]["deaths"] = 0
            print("Not Found", cur_county["properties"], "\n")

Subbing in 0500000US51159 for 0500000US51760
Subbing in 0500000US29189 for 0500000US29510
Subbing in 0500000US51059 for 0500000US51600
Subbing in 0500000US51515 for 0500000US51019
Subbing in 0500000US51620 for 0500000US51067
Not Found {'geo_id': '0500000US24005', 'county': '005', 'name': 'Baltimore', 'lsad': 'County', 'censusarea': 598.302, 'state': '24', 'statename': 'Maryland', 'stateabbr': 'MD', 'population': 828431, 'land_area': '598.30', 'housing_units': '335622', 'population_density': 1384.6414842052484, 'cases': 0, 'deaths': 0} 

Subbing in 0500000US51770 for 0500000US51161


### Breakdown NYC into counties by proportional estimate

In [38]:
dict_cases['0500000US36081']

('New York', 'Queens', 0, 0)

In [39]:
# This is based on an estimate on 3/21/20 in which NYC had 5687 cases broken down as follows
# # Kings - 1750, Queens - 1514, New York 1402, Bronx 736, Richmond 285
proportion_dict = {
    #code:proportion of NYC cases
    #queens
    '0500000US36081':.26622121,
    #kings
    '0500000US36047':.30771936,
    #Bronx
    '0500000US36005':.12941797,
    #Richmond (Staten Island)
    '0500000US36085':.05011429,
    #New York (Manhattan)
    '0500000US36061':.24652717,
}

# #get cases that were all aggregated in NY county
state, county, ny_cases, ny_deaths = dict_cases['0500000US36061']
# #split them up into the correct counties
queens_cases = dict_cases['0500000US36081'][3]
# sometimes they include each county, sometimes they just include NY county, so we check and if 
# queens cases are 0 we run the proportion
if queens_cases == 0:
    for geo_id, proportion in proportion_dict.items():
        county_cases = int(proportion*ny_cases)
        county_deaths = int(proportion*ny_deaths)
        state, county, _, _ = dict_cases[geo_id]
        dict_cases[geo_id] = (state, county, county_cases, county_deaths)

In [40]:
for cur_county in county_json["features"]:
    county_id = cur_county["properties"]["geo_id"]
    if county_id in proportion_dict.keys():
        cur_county["properties"]["cases"] = dict_cases[county_id][2]
        cur_county["properties"]["deaths"] = dict_cases[county_id][3]

In [41]:
# dict_cases['0500000US36061']

In [42]:
# _, _, ny_cases, ny_deaths = dict_cases['0500000US36061']
# #New York (manhattan) '0500000US36061'
# #queens '0500000US36081'
# #kings (brooklyn) '0500000US36047'
# #bronx '0500000US36005'
# #staten island '0500000US36085'

## Add in percapita stats

In [43]:
for cur_county in county_json["features"]:
    props = cur_county["properties"]
    per_cap = props["population"]/100000
    props["pc_cases"] = props["cases"]/per_cap
    props["pc_deaths"] = props["deaths"]/per_cap

## Add Centroids

In [44]:
with open(data_folder/"countyCentroids.json", 'r') as f:
    county_centroids = json.load(f)

In [45]:
county_centroids["features"][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01029',
  'STATE': '01',
  'COUNTY': '029',
  'NAME': 'Cleburne',
  'LSAD': 'County',
  'CENSUSAREA': 560.1},
 'geometry': {'type': 'Point',
  'coordinates': [-85.5187931476936, 33.674542689931684]}}

In [46]:
for county in county_json["features"]:
    for county2 in county_centroids["features"]:
        if county["properties"]['geo_id'] == county2["properties"]['GEO_ID']:
            county["properties"]["lat"] = county2["geometry"]["coordinates"][0]
            county["properties"]["long"] = county2["geometry"]["coordinates"][1]

## Add Risk Data

In [47]:
#111 is to convert degrees to kilometers
def get_distance(c0, c1):
    lat_dist = abs(c0[0])-abs(c1[0])
    lng_dist = abs(c0[1])-abs(c1[1])
    distance = 111 * math.sqrt(lat_dist**2 + lng_dist**2)
    return distance

### Local Risk

In [48]:
def calc_county_local_risk(props):
    cases = props["cases"]
    try:
        population = props["population"]
    except KeyError:
        print(props['name'], props['statename'])
        raise KeyError
    if cases == 'NaN': 
        print('NaN cases found')
        cases = 0
    return cases/(population/100000) if population != -1 else -1

In [49]:
dict_cases['0500000US01001']

('Alabama', 'Autauga', 0, 0)

In [50]:
for cur_county in county_json["features"]:
    cur_county["properties"]["risk_local"] = calc_county_local_risk(cur_county["properties"])

### Neighbor Risk

In [51]:
def calc_county_neighbor_risk(risks):
    total_neighbor_risk = 0
    num_risks = len(risks)
    for risk in risks.values():
        distance, neighbor_cases, neighbor_pop = risk.values()
        neighbor_density = float(neighbor_cases)/(neighbor_pop/100000)
        total_neighbor_risk += float(neighbor_density)*(2**(-1*math.log2(float(distance))))
    return total_neighbor_risk

In [52]:
def get_county_all_neighbor_risk(props):
    risk_details = {}
    for cur_county in county_json["features"]:
        neighbor_props = cur_county["properties"]
        neighbor_risk = get_county_neighbor_risk(props, neighbor_props)
        if(neighbor_risk != {}): risk_details[neighbor_props["geo_id"]] = neighbor_risk
    population = props.get("population", 1000000)
    props["risk_nearby"] = calc_county_neighbor_risk(risk_details)
    props["risk_total"] = props["risk_nearby"] + props["risk_local"]

In [53]:
MAX_DISTANCE = 250
def get_county_neighbor_risk(props, neighbor_props):
    geoID1 = props["geo_id"]
    geoID2 = neighbor_props["geo_id"]
    centroid = [props["lat"], props["long"]]
    centroid_neighbor = [neighbor_props["lat"], neighbor_props["long"]]
    distance = get_distance(centroid, centroid_neighbor)
    neighbor_cases = neighbor_props["cases"]
    neighbor_population = neighbor_props["population"]
    if(geoID1 == geoID2 or distance > MAX_DISTANCE or neighbor_cases == "NaN"): 
        return {}
    else:
        return {"distance":distance, "cases":neighbor_cases, "pop":neighbor_population}

In [54]:
for cur_county in county_json["features"]:
    get_county_all_neighbor_risk(cur_county["properties"])

In [55]:
county_json["features"][1002]

{'type': 'Feature',
 'properties': {'geo_id': '0500000US19163',
  'county': '163',
  'name': 'Scott',
  'lsad': 'County',
  'censusarea': 458.089,
  'state': '19',
  'statename': 'Iowa',
  'stateabbr': 'IA',
  'population': 173283,
  'land_area': '458.09',
  'housing_units': '71835',
  'population_density': 378.2728284834858,
  'cases': 1,
  'deaths': 0,
  'pc_cases': 0.5770906551710208,
  'pc_deaths': 0.0,
  'lat': -90.62324265750517,
  'long': 41.63709282155562,
  'risk_local': 0.5770906551710208,
  'risk_nearby': 1.2078852610165058,
  'risk_total': 1.7849759161875265},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-90.898373, 41.771392],
    [-90.473617, 41.773465],
    [-90.317041, 41.729104],
    [-90.334525, 41.679559],
    [-90.343162, 41.648141],
    [-90.34165, 41.621484],
    [-90.39793, 41.572233],
    [-90.461432, 41.523533],
    [-90.474332, 41.519733],
    [-90.499475, 41.518055],
    [-90.605937, 41.494232],
    [-90.655839, 41.462132],
    [-90.737537, 41.450127]

In [56]:
county_json["features"][0]["properties"].keys()

dict_keys(['geo_id', 'county', 'name', 'lsad', 'censusarea', 'state', 'statename', 'stateabbr', 'population', 'land_area', 'housing_units', 'population_density', 'cases', 'deaths', 'pc_cases', 'pc_deaths', 'lat', 'long', 'risk_local', 'risk_nearby', 'risk_total'])

## Export

In [57]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

with open("../../data/counties.json", 'w') as f:
    json.dump(county_json, f, cls=NpEncoder)
    
with open("../../data/countyData.js", 'w') as f:
    f.write("let countyData = ")
    json.dump(county_json, f, cls=NpEncoder)

In [58]:
!jupyter nbconvert --to script 10_counties.ipynb

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern '10_counties.ipynb' matched no files
